# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [12]:
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
        
    last_height = new_height
    

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [13]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [14]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 北市精華地169人共有　百年家族超猛！直接成立建設公司花15年都更整合

▲六○年代是黑松的全盛時期，每天生產十萬打汽水，據說只要能成為黑松的經銷商，連走路都有風。（圖／黑松提供）
圖、文／CTWANT
近年雙北不少困難度極高的都更案，紛紛接力動工拆建，但多數人不知道的是，其中位於台北市南西商圈的豪宅建案「中山松悅」，是近百年汽水品牌「黑松」張家的起家厝，為了都更，張家還特別成立「詔安建設」，由第4代女婿黃茂霖擔任總經理，花了15年整合169位地主。
▲黑松前董事長張道宏﹙左﹚與現任董事長張斌堂﹙右﹚為叔姪關係，2人在10年前的經營權之爭，也讓家族故事浮上檯面。（圖／報系資料庫）
黃茂霖進一步透露，黑松家族在台北還有很多土地，都像「中山松悅」一樣是多人共同持有，接下來會一塊塊處理，也不排除購地推案。他強調，「詔安建設」是為了完成長輩心願而成立，屬於黑松家族自己的公司，而上市公司黑松，並非百分之百為張家所有，兩間公司之間沒有直接關聯。
「我們黑松家族就是這麼可怕！」一名張家後代透露，黑松家族發達後，土地遍布台北市，不僅張家祖輩曾坐擁信義區大批土地，現今復興南路上的「微風廣場」也是向黑松承租使用。在眾多土地中，黑松家族最寶貝「詔安故居」這塊起家地。
▲現今微風廣場的所在地，前身是黑松1937年以6萬日圓買地興建的黑松汽水中崙廠，目前每年可為黑松創造近5億元租金收益。
「詔安故居」日據時期位於「大正町」火車站旁，該火車站光復後改名「長安站」；隨著張家人丁興旺，原本是三合院的「詔安故居」，向外擴建形成二進院落的大宅院，成為鐵路旁的醒目地標，「那時看到張家古厝，就知道台北車站快到了！」當地一名耆老說。
▲黑松特別在微風廣場設立「黑松世界」博物館，保留這段歷史與品牌文化。（圖／報系資料庫）
「今日的市民大道與南京東路之間的九條通，日據時期屬於大正町，與一旁的御成町是高級住宅區、日本人最多的地方，經過百年演變，從詔安厝、御成町、大正街，最後成為現在的長安西路。」台北市中山區民安里里長羅孝英表示，現在的「台北光點」是前美國駐台北領事館，台北市政府原本位於長安西路，也就是現在的「台北當代藝術館」，當時周邊是台北最繁榮之處。(待續)
▲

▲美股收漲131點。（圖／路透社）
文／中央社紐約23日綜合外電報導
美國總統川普表示與中國的第一階段貿易協議未變，加上歐美數據增加經濟迅速復甦的希望，投資人不理會美國許多地區新冠肺炎病例急遽增加，美股今天收紅，那斯達克指數再創新高。
道瓊工業指數上漲131.14點或0.5%，以26156.1點作收。
標準普爾指數上揚13.43點或0.43%，收在3131.29點。
科技股那斯達克指數上漲74.89點或0.74%，收10131.37點。
----------------------------------------------------------------------

[10] 大樂透端午加碼「百萬大紅包」開出14組　二獎247萬一注獨得

記者林彥臣／綜合報導
本期大樂透頭彩1億元，開出得得獎獎號為01、02、16、22、27、45，特別號13，雖然頭彩槓龜沒開出，但是端午加碼的百萬大紅包開出14組注獨得，目前百萬大紅包還有48組。
本期大樂透頭彩獎金不足1億，台彩公司加碼補足1億元彩金，雖然本期頭獎槓龜沒有開出，但是二獎一注獨得，得獎金新台幣247萬2160元。端午節加開的62組百萬大紅包，本期開出14組一注獨得，尚有48組100萬獎項未開出，將於次期（26日）繼續加開。
►省油80%，瘦身於無形
----------------------------------------------------------------------

[11] 台水宣佈今年水費凍漲　紓困水費減免「全台90萬戶受惠」

▲台水公司109年股東常會於今日在台灣自來水公司總管理處大禮堂舉行，由董事長胡南澤主持。（圖／台水提供）
記者林淑慧／台北報導
新冠肺炎疫情衝擊經貿發展，台灣自來水公司今（23）日表示，疫情衝擊百業，今年水費將持續凍漲，另外配合政府紓困措施提出水費減免，據統計全台約有90萬戶受惠。
台水公司109年股東常會於今日在台灣自來水公司總管理處大禮堂舉行，由董事長胡南澤主持，出席股權數達93.49 ％。
胡南澤表示，台水存在許多經營上的困境，如極端氣候威脅、新水源開發不易、飲用水水質標準趨嚴、許多管線老舊待汰換、水價長期未能合理反映成本等，均不利於自來水事業的經營。
特別是工業用水大戶配合政策使用再生水，如中鋼及中鋼鋁簽訂鳳山再生水廠再生水之用水量每日約4.

----------------------------------------------------------------------

[21] 振興三倍券首波將印製1200萬份　經濟部：視領用情況動態調整

▲實體「振興三倍券」分成5張200元及4張500元的發放模式。（圖／行政院提供）
記者林淑慧／台北報導
振興三倍券七月即將登場，網傳不實消息又起，經濟部今（23）日澄清表示，振興三倍券第一階段將由中央印製廠印製1,200萬份，再視領用情況動態調整。7月1日至7月7日民眾可持健保卡至超商或振興三倍券官網(https://3000.gov.tw/)預訂；於7月15日起至郵局、四大超商、全聯、美廉社、屈臣氏及康是美等超市藥妝通路領取。
網傳「海洋大學黃○○教授說明振興券的妙用」，稱沒有人知道振興三倍券印製及領取數量、預算、將未領用振興三倍券提供店家兌現，甚至稱三倍券預算有一兆五百億等內容，經濟部今天發佈書面聲明表示，上述都是錯假訊息。
▲經濟部今（23）日表示，台北市、新北市、桃園市、新竹縣及花蓮縣等地方政府，都已配合三倍券推出自家的加碼優惠。（圖／經濟部提供）
官員指出，振興三倍券的預訂及領取過程皆採實名登記，每位國民只有一次領取機會，印製及領取數量皆有嚴格控管，絕不可能有將未領用的振興三倍券提供店家兌現之情事。
另外，紙本振興三倍券印製採鈔券規格，由中央印製廠印製，採鈔券之防偽等級，有多項防偽特點，如隱藏字、微小字、螢光纖維絲等，每一張皆有獨一序號，可以嚴格管控印製數量。振興三倍券比照現鈔，由中央印製廠運鈔車及中華郵政運送，搭配警力全程支援護送至分裝點及統倉，在每個交接環節皆嚴格進行點收，確實掌握庫存數量，並由警力派駐看守統倉，以保障振興三倍券安全存放。
振興三倍券預估總經費則為505億元，其中印製、配送及兌付等行政成本約9億元，占總經費1.8%。經濟部推估，振興三倍券自7月15日啟用後，搭配暑假旅遊旺季及業者加碼活動，預估可創造1,000億元消費效益。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[22] 5月新增房貸平均利率1.349%　續創歷史新低紀錄！

▲5月新增房貸利率創歷史新低 。 （圖／記者林裕豐

▲新台幣收盤收29.635元。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（23）日收盤收29.635元，升值2.5分，成交金額6.97億美元。
新台幣匯率今天開盤升值3分，以29.630元兌1美元開出，盤中最高價位29.552元，最低價則是29.635元兌1美元。
新台幣匯率終場收29.635元，升值2.5分。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[33] 五月餐飲業回溫「營收月增29%」 經濟部：創史上最大月增幅

▲經濟部統計處今（23）日發佈批發、零售及餐飲業動態調查統計。（圖／經濟部提供）
記者林淑慧／台北報導
國內疫情趨緩，民眾生活逐步回歸常軌，加上五一勞動連假及母親節聚餐效應，經濟部今（23）日發佈批發、零售及餐飲業動態調查統計，5月餐飲業營業額619億元，較4月大幅彈升29.0%，為開辦本項調查以來最大月增幅。
經濟部統計處副處長王淑娟指出，5月餐飲業營業額與上年同月相比，仍減少8.7%，其中餐館業年減7.1%，惟較2至4月的二位數減幅明顯收斂，漸次走出谷底陰霾，飲料店業年減幅度亦縮減至8.3%；外燴及團膳承包業則年減32.8%，主因各國持續邊境管制衝擊空廚營收所致。
零售業表現方面，部分業者反因疫情而逆勢成長，超市、便利商店、量販店、電子購物及郵購業分別年增8.5%、7.3%、14.2%及22.2%，營業額皆創歷年同月新高。
▲經濟部統計處副處長王淑娟。（圖／記者林淑慧攝）
隨國內疫情趨緩，消費人潮逐漸回流，天氣轉熱亦致冷氣銷售增溫，加上油價回彈，汽柴油銷量攀升，5月零售業營業額3,008億元，較4月增加8.3%；與上年同月相比，則仍減少5.8%，主因國際觀光客仍大減，消費力道尚未完全恢復所致，惟減幅已明顯縮小。
其中，百貨公司年減6.3%、其他綜合商品零售業(包括免稅店等)年減39.0%、食品飲料及菸草零售業年減12.9%、家用器具及用品零售業年減19.8%；燃料零售業亦因汽柴油價格較上年同月大幅滑落而年減35.4%；汽機車零售業年減6.1%，主因部分品牌進口車因國外停工，新車到港數下滑，加上去年新車款上市比較基數高，以及電動機車買氣降溫所致。
批發業表現方面，5月批

▲台股終場收在11612點。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
台股今 (23) 日大盤盤中加權指數一度因市場擔憂中美貿易衝突再起而翻跌，但隨後買盤加湧入，指數站上 11600 點，力守盤面高檔震盪格局，終場收在11612點，上漲39點或0.34%，成交量2041.48億元。
蘋概三王中，台積電（2330）以315元作收、上漲0.96%；鴻海（2317）以80.2元作收、上漲1.65%；股王大立光（3008）以4010元作收、下跌0.25%。
大型權值股中，除了大立光和聯發科下跌外，其他都上漲，台塑四寶則上漲0.12%~1.25%，國泰金上漲0.48%，富邦金上漲0.92%。
今日上市各類股以油電類股表現最佳，上漲1.21%，表現較差者為電器類股，下跌1.98%。 
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[44] 我節電我驕傲！台電：今年目標節省超過20億度電

▲台電節電活動年度宣傳大使、自家女排隊長張秝芸以招牌pose向大家分享節電心得。（圖／台電提供）
記者林淑慧／台北報導
隨著6月用電屢創今年及歷年同月新高，台電今 (23)日宣布年度節電活動正式起跑，今年目標全台節電超過1%，也就是超過20億度，但因節電難度年年提升，台電今年也持續將目標放在年輕人，以生活美學觀念、活動來呼籲節電。
台電今日上午在台電大樓舉辦2020節能減碳啟動記者會，由台電總經理鍾炳利與年度宣傳大使、自家女排隊長張秝芸（秝音同立）及22位節電績優得獎學校共同宣誓活動起跑。台電今年延續去年「我節電、我驕傲」活動主軸，主打「省下幾度，就是我們的生活態度」，能讓節電成為引以為傲的生活方式和態度。
根據台電統計，全國各國中小學去年節電量逾2560萬度，共計發出近1700萬元節電獎勵金。台電也首度於會中公布「校園節電績優地圖」大數據調查，從2019年各縣市國中小學「節電率」最高行政區中選出「節電量」最高的國中小學給予獎勵，因而共有22個節電冠軍學校獲獎。
其中，台北市台北歐洲學校以節電逾33萬度奪冠，可說是節電校園「冠軍中的冠軍」，其次則為桃園市清華高中（附國中部）節電12.1萬度及嘉義市興華高中節電10.6萬度分別拿下亞軍及季軍。
台電說

▲國際疫情再度升溫，黃金已出現避險買盤。（圖／路透）
記者王曉敏／綜合報導
全球再陷新一波疫情疑慮，使避險需求近期再度提升，黃金現貨價格上漲1%至每盎司1760.55美元；8月黃金期貨收盤上漲0.8%至每盎司1766.4美元，為4月14日以來最高收盤價，盤中一度觸及每盎司1779美元，接近2012年來高點。
截至今日上午10點30分，8月黃金期貨下跌0.22%至每盎司1762.6美元。
RJO期貨公司資深市場策略師Bob Haberkorn表示，黃金已出現避險買盤，「全球新冠肺炎確診病例增加導致金價突破每盎司1750美元的水準。如果今天收盤價高於每盎司1765美元，那達到1800美元的關卡就不算太遠了。」
世界衛生組織（WHO）上周日公布全球創下至今最高單日新增確診紀錄，黃金價格隨即提振。 ThinkMarkets市場分析師Fawad Razaqzada表示，疫情造成的經濟損失及對第二波疫情的擔憂，無疑使作為資金避風港的黃金獲得了提振，「黃金也從各國央行挹注大量資金進入市場中獲得了支撐。」
高盛（Goldman Sachs）上周五發布研究報告，將12個月的黃金價格預測上調至每盎司2000美元，並維持了2020年12月的看多黃金的交易建議。
其他貴金屬方面，截至稍早，7月白銀期貨下跌0.6%至每盎司17.795美元；7月鉑金期貨下跌0.91%至每盎司836美元；9月鈀金期貨下跌0.3%至每盎司1,936.2美元。
----------------------------------------------------------------------

[55] 小股東提出「鴻海股東全民化」　董座劉揚偉：毛利率提升至10%可望增加股息

▲鴻海股東會由董事長劉揚偉主持。（圖／記者姚惠茹攝）
記者姚惠茹／新北報導
鴻海（2317）股東會今（23）日由董事長劉揚偉主持，現場有小股東提出「鴻海股東全民化」，不要讓外資佔的比例太高，希望讓股東多領一點股息，增加全民持有鴻海股票，對此劉揚偉回應，當毛利率可以持續提升到10%，過程中就會逐漸反饋給股東，希望未來賺更多錢回饋股東。
鴻海小股東表示，現在鴻海的股票外資佔的比例太高，已經超過600萬張，所以造成股價上沖下洗，所以希望可以做到「鴻海股東全民化」，而且現在的未分配盈餘太多，希望藉由讓股東多領一點股息，增加全民持有鴻海股票。

▲WWDC 2020公開iOS 14、iPadOS 14、WatchOS 7、tvOS 14以及代號為 Big Sur 的 MacOS。（影片／取自YouTube）
特約記者洪聖壹／綜合報導
Apple公司稍早史無前例地舉辦線上的開發者大會（WWDC 2020），發表地點依然選在 Apple 公司的賈伯斯劇院（Steve Jobs Theater），發表了新版iOS 14、iPadOS 14、WatchOS 7、tvOS 14以及代號為 Big Sur 的 MacOS。
一、iOS 14：
針對iPhone 的新版作業系統，蘋果公司以「讓手機用起來更簡單」為前提來提供多項設計，全新設計的 App Library 的應用，讓用戶可以在分類好的 Apps 資料夾當中，找到自己的應用程式。
全新的設計的Widgets可以把天氣、運動紀錄加入主螢幕，並且加入全新畫中畫功能，讓用戶可以暫時縮小影片視窗，同時作筆記、通話。至於Siri換上新的動畫，啟用時不再遮擋螢幕，而且加入了輸入法聽寫功能，並且可以辨識語音，提供語音訊息。
全新的翻譯功能可以在英文、中文、法文、德文、西班牙文、義大利文、日文、韓文、阿拉伯文、葡萄牙語、俄羅斯文等11種語言當中互換。訊息功能上面，加入了20種全新memoji圖像以及3種新的動態貼圖，而且新增了類似WeChat、LINE的"@“功能，蘋果公司稱為mention（提到某人），並在同一時間新增類似Twitter的特定用戶對話。
接下來可能多數用戶暫時還用不到的3D、實景地圖，以及需要很有錢才能拿手機當車鑰匙的CarPlay功能。倒是台灣推行很多年的 NFC 跟 QRCode，現在 iOS 14 添加的 App Clip 功能，可以讓手機透過感應或者掃描，就可以快速付款買餐、租車、拿取優惠券。這個App Clip 有點類似縮小版應用程式，大小約在10MB，讓用戶能夠在不需要下載應用程式的前提下使用某些特定功能，當然，如果用戶喜歡店家提供的應用程式，可以直接跳轉到 AppStore下載完整版。
二、iPadOS 14：
這次 iPadOS 14 加入全新的桌面主題設計，而且在使用 Siri、Facetime 時，不再會遮擋整個螢幕，該功能也將支援第三方應用程式，好比說Skype。
再來，此版本的另一項重大更新是Apple稱其為“通用搜尋”的“徹底重建”引

記者姚惠茹／新北報導
鴻海（2317）今（23）日將在土城總部召開股東常會，由董事長劉揚偉首次主持大會，而這也是劉揚偉掌舵鴻海屆滿週年交出成績單的日子，從鴻海Foxconn 3.0的轉型升級計畫，全力聚焦「3+3」產業發展，再到帶領集團走過對抗新冠肺炎的過程，這張成績單受到小股東矚目。
鴻海創辦人郭台銘去年股東會交棒董事長職務，如今劉揚偉掌舵鴻海集團屆滿週年，鴻海正由內而外的持續改變，從公司治理強調內外溝通的一致，為員工的身心健康發展改造土城總部的健身中心，再為投資人召開每半年一次的法說會，真正做到當初承諾的透明管理。
劉揚偉積極帶領鴻海啟動Foxconn 3.0的轉型升級計畫，全力聚焦「3+3」產業發展，陸續宣布與歐洲汽車大廠飛雅特簽定MOU，並與裕隆旗下華創合資進軍電動車市場，還有攜手工業電腦廠凌華布局機器人市場，加速布局電動車、數位醫療及機器人三大產業。
劉揚偉面臨新冠肺炎疫情的衝擊，超前部署的在台灣成立「防疫指揮中心」，並設立美洲、歐洲、亞太與中國大陸四大區域的防疫體系，更親自坐鎮召開每日防疫連線會報，就在防疫視同作戰的情況之下，提前在3月中旬完成中國大陸各廠區的全面復工，並因應產能作好調度安排。
鴻海股東會每年都有不少小股東參與，今年由劉揚偉首次主持大會，這張掌舵鴻海屆滿週年的成績單更是受到小股東矚目，同時市場高度關注股東會中釋出今年下半年市況、營運展望及Foxconn 3.0轉型升級計畫的進程。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[73] 那斯達克再破萬點「刷新歷史紀錄」　美股三大指數齊揚

▲蘋果開發者大會（WWDC）創造新話題。（圖／取自YouTube／Apple）
記者張靖榕／綜合外電報導
華爾街持續加注各家具潛力的科技股，加上蘋果WWDC再次掀起話題，包括蘋果、Adobe、亞馬遜等科技大公司，皆創下歷史新高，美股三大指數齊漲外，那斯達克指數也再次衝破萬點，刷新歷史紀錄，並締造連續7個交易日收紅。
道瓊工業指數終場上漲153.50點，或0.59％，收在26024.96點；標準普爾指數上漲20.12點，或0.65%，以3117.86點作收；那斯達克指數上漲110.35點，或1.11％，以10

記者柯振中／綜合報導
隨著新冠肺炎疫情趨緩，台灣房市逐步解封，不少民眾趁著假日預約看房。經過業者統計，最近周末的看屋人潮與4月相比，人數多了約2成，且民眾看的產品類型與以往有別，解封後民眾瀏覽的產品以「透天」居冠。
根據《591新聞網》報導，房仲業者統計，6月13日、14日的看屋量與4月的周末相比，平均成長2成，可能是疫情趨緩，讓看屋人潮重新回歸。
依據相關數據，今年2月到6月之間，民眾在買房網站的瀏覽物件以透天為大宗，比例高達44.5％，接下來則是電梯大樓，比透天的瀏覽量少了近10％。
與去年相比，當時民眾喜好偏向電梯大樓，瀏覽比例高出透天約3％左右，可能是疫情影響，民眾的看屋喜好也開始改變。
以各縣市看屋狀況分析，台南市、台中市與桃園市的民眾最常瀏覽透天物件。平均下來的話，台南市能以較少的金額購買到較大的坪數，再來則是桃園市。
房仲公司分析，受到近年來人口比例影響，多數購屋人喜愛購買社區大樓，不過疫情衝擊後，也許是為了保持社交距離，民眾漸漸開始對透天類型的物件有興趣，但也是在總價相近的狀況下，一般以中南部居多。
此外，依據先前統計，近期有不少台商回流，鎖定房屋物件，台商、換屋族佔了房市交易量的7成，規劃回台發展、投資的民眾、台商有增加的趨勢。
----------------------------------------------------------------------

[84] 彰銀工會明開記者會　要求台新金「要賣趕緊賣！嘜擱亂啊！」

▲彰銀工會明日將召開記者會，要求台新金放手彰銀。（圖／記者吳靜君攝）
記者吳靜君／台北報導
彰銀股東會上週五（19日）改選董事，但是選完才是紛爭的開始，彰銀工會早在改選後對於台新金所派出的獨立董事及台新金的策略有所質疑。彰銀工會今（22）日表示，全金聯與彰銀工會將明（23）日將召開記者會並將提出主張，台新金並無當選法人董事，要求台新金應依改選結果盡快放手彰銀。
彰銀上周五董事改選，公股銀拿下五席一般董事、台新金則是拿下一席一般董事（自然人）以及三席獨立董事，等於獨立董事掌握在台新金手上，而改選後的首次董事會就出現爭議，台新金質疑常董的資格必須是自然人而非法人董事，使得常務董事、董事長差一點難產。
彰銀工會對於台新金與「傾向」台新金的獨立董事表達三點看法，一，獨立董事應該具獨立性，應為全體股東利益發聲，並非傾向台新

▲國泰金今日舉行「經濟情勢與金融情勢」展望發表。（圖／記者吳靜君攝）
記者吳靜君／台北報導
國泰金今（22）日舉辦第二季台灣「經濟暨金融情勢」發表會，對於政府將發放振興三倍券的看法，國泰金與台灣大學合作計畫主持人徐之強表示，預估可貢獻台灣經濟成長率（GDP）0.1個百分點到0.2個百分點，整體而言今年的經濟成長率估為1.8%。
徐之強表示，台灣每年的GDP（國內生產毛額）大約是20兆元，預估今年的經濟成長率大約是1.8%，較原先預估的2.0%，再下調0.2個百分點。主要是許多國家被迫封城鎖國，一度造成景氣及動以及金融市場大幅緊縮，所幸各國都推出大規模的貨幣寬鬆及財政政策，有效緩和緩和金融情勢緊張的壓力，國泰金研判，全年的經濟成長率有八成的機會落在1.0到1.8%之間，所以今年經濟成長率還有機會「保一」。
國泰金表示，新冠肺炎（COVID-19）疫情陰影之下，台灣上半年表現低迷，不過隨著各國陸續解封，海外需求逐漸改善，搭配寬鬆貨幣政策，第三季台灣的經濟成長率就有機會可以由負轉正。
也因為台灣疫情控制良好、防疫措施逐漸寬鬆，民間消費可帶動下半年景氣改善，台灣的央行下半年不需要再降息，但後續仍需要依據疫情的發展而定。
而根據國泰金所評估，政府所發放的振興三倍券對今年的經濟成長仍有貢獻，大約貢獻0.1個百分點到0.2個百分點，然而，國泰金所預估今年的1.8%，已經考慮了振興三倍券的貢獻。
但國泰金強調「最壞情況已過」，隨著政府陸續祭出振興措施，下半年將是「隧道中看到曙光」。
----------------------------------------------------------------------

[94] 智伸科營收獲利「雙創新高」！每股配息2.5元　5大經營方針成營運動能

▲智伸科總經理林恩道。（圖／智伸科提供）
記者楊絡懸／桃園報導
汽車動力暨安全零組件大廠智伸科（4551）22日舉行股東會，會中承認2019年度營業報告書、財務報表和各項討論事項，並通過2019年盈餘分配案，決議配發每股現金股利2.5元，期將經營成果與全體股東分享。
智伸科股東會進行全面改選董事，依公司章程規定，票選出董事9席（含獨立董事3席），新當選董事包括林正盛、六方精機股份有限公司、盧經緯、何瑞正、林良雄、林恩道；3席獨立董事則由蔡佳瑜、楊翔宇、辜清德擔任。
股東會結束後隨即